In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import math

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler,OneHotEncoder
from sklearn.metrics import mean_squared_error

from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD

In [65]:
df = pd.read_csv('../raw_data/data.csv').drop(columns='Unnamed: 0')

df

,date,symbol,Dividend,volume,fed_funds_rate,gdp,debt_to_equity,eps,return_on_equity,quick_ratio,operating_ratio,inventory_turnover,stock_price,sa_neu,sa_pos,sa_neg
0,2018-07-01,HPQ,0.00,6.845513e+06,1.91,4689.905000,-20.385399,0.556258,-0.498019,0.614953,0.925134,1.953374,19.810163,NaN,NaN,NaN
1,2018-08-01,CCL,0.50,3.182053e+06,1.91,4717.323667,0.693559,2.417847,0.069238,0.168329,0.692598,7.303704,55.871806,8.332660e-01,1.622590e-01,4.474991e-03
2,2018-08-01,FDS,0.64,2.294333e+05,1.91,4717.323667,1.699082,1.801999,0.130867,1.920293,1.234522,27.010998,203.856165,8.916588e-01,9.585273e-02,1.248847e-02
3,2018-08-01,FDX,0.00,1.211117e+06,1.91,4717.323667,1.706984,3.150943,0.043551,1.261503,0.378313,7.585086,224.475233,7.502943e-01,1.412142e-03,2.482935e-01
4,2018-08-01,HPQ,0.00,6.493083e+06,1.91,4717.323667,-25.713635,0.679721,-0.769470,0.613254,0.927927,1.998739,20.428785,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13033,2023-03-01,XYL,0.00,1.532100e+06,4.65,5053.623333,1.216042,0.548780,0.027669,1.441098,0.931630,1.052509,100.078591,7.908352e-10,1.000000e+00,6.626421e-09
13034,2023-03-01,YUM,0.00,1.474093e+06,4.65,5053.623333,-1.655231,1.067616,-0.034192,0.709968,0.435866,0.911765,126.797676,1.658575e-07,9.999998e-01,1.465031e-07
13035,2023-03-01,ZBH,0.24,1.147900e+06,4.65,5053.623333,0.747768,1.110315,0.019135,0.929235,0.820884,0.223442,124.411347,1.128311e-02,5.903665e-01,3.983504e-01
13036,2023-03-01,ZION,0.00,5.970143e+06,4.65,5053.623333,16.085841,1.377448,0.039352,0.296107,0.615894,-17.000000,38.069854,9.987674e-01,4.632685e-07,1.232253e-03


## Todo
1. ~~FFil SA~~

2. train-test-split
3. Standard scale float,
5. Create seq
6.  Model, embedding layer w mse and rmse


In [88]:

df['date'] = pd.to_datetime(df['date'])
df.sort_values(by=['symbol', 'date'], inplace=True)

data = df.copy()

data


,date,symbol,Dividend,volume,fed_funds_rate,gdp,debt_to_equity,eps,return_on_equity,quick_ratio,operating_ratio,inventory_turnover,stock_price,sa_neu,sa_pos,sa_neg
884,2019-01-01,A,0.000,2.196933e+06,2.40,4584.814000,0.777601,1.584906,0.100079,2.793607,0.847551,0.883614,66.795341,0.499995,5.000018e-01,0.000003
1190,2019-02-01,A,0.000,1.866410e+06,2.40,4640.884333,0.771797,1.248688,0.078305,2.803204,0.853717,0.877738,73.297129,0.444752,5.552406e-01,0.000008
1496,2019-04-01,A,0.164,1.807587e+06,2.42,4753.025000,0.760390,0.574132,0.035512,2.822004,0.866503,0.866058,77.022864,NaN,NaN,NaN
1808,2019-05-01,A,0.000,3.190033e+06,2.39,4767.423667,0.778289,0.587924,0.037007,2.360550,0.866414,0.871327,71.172318,0.452402,2.647111e-01,0.282886
2124,2019-07-01,A,0.164,2.552590e+06,2.40,4796.221000,0.816937,0.616129,0.040236,1.724414,0.866242,0.881818,69.973080,0.712765,1.227886e-03,0.286007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11607,2022-10-01,ZTS,0.325,1.883153e+06,3.08,5157.178000,2.078945,1.087570,0.110626,1.529853,0.660490,0.268594,148.505972,0.334795,4.938512e-03,0.660266
11963,2022-11-01,ZTS,0.000,2.671823e+06,3.78,5088.392000,2.230832,1.040846,0.107697,1.581289,0.675436,0.273559,145.341229,0.999994,2.939670e-07,0.000006
12321,2023-01-01,ZTS,0.375,2.219257e+06,4.33,4950.820000,2.278037,1.060347,0.110794,1.700570,0.677467,0.261133,153.280092,0.999962,5.302767e-06,0.000033
12680,2023-02-01,ZTS,0.000,2.336277e+06,4.57,5002.221667,2.168894,1.126873,0.116852,1.796913,0.664570,0.244813,165.454036,0.501850,4.979981e-01,0.000152


In [89]:
col_tofill = ['sa_neu','sa_pos','sa_neg']

for col in col_tofill:
    data[col] = data.groupby(data['date'].dt.to_period('6M'))[col].ffill()


In [91]:
data.isnull().sum()/len(df)*100

date                  0.000000
symbol                0.000000
Dividend              0.000000
volume                0.000000
fed_funds_rate        0.000000
gdp                   0.000000
debt_to_equity        0.000000
eps                   0.000000
return_on_equity      0.000000
quick_ratio           0.000000
operating_ratio       0.000000
inventory_turnover    0.000000
stock_price           0.000000
sa_neu                1.288541
sa_pos                1.288541
sa_neg                1.288541
dtype: float64

## Scaling

In [120]:
# import gensim
# from gensim.models import Word2Vec
# # # def clean_tickers(symbol):
# # #     return symbol.lower()

# # tickers = [tickers.lower() for tickers in data['symbol']]
# # corpus = [[word] for word in tickers]
# # type(np.unique(np.array(tickers)))
# # model = Word2Vec(sentences=corpus, min_count = 1,vector_size = 100, window = 5)

# # model.wv['aapl']
# # model.wv.most_similar('psx')




In [99]:
#Numerical Scaler

num_col = data.select_dtypes(include='float')

num_scaler = StandardScaler()

data[num_col.columns] = num_scaler.fit(num_col)

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [49]:
data_grouped = df.groupby('symbol')

sequence_length = 12  # Define the sequence length
step_size = 1  # Define the step size (e.g., move one month at a time)

sequences = []
labels = []

for symbol, group in data_grouped:
    data = group.drop(columns=['date', 'stock_price'])  # Exclude date and target variable
    target = group['stock_price'].values  # Target variable (stock_price)

    for i in range(0, len(data) - sequence_length + 1, step_size):
        sequence = data[i:i+sequence_length].values
        label = target[i+sequence_length-1]
        sequences.append(sequence)
        labels.append(label)

# Convert sequences and labels to NumPy arrays
sequences = np.array(sequences)
labels = np.array(labels)

## Model Predicting

In [17]:
def plot_predictions(test,predicted):
    plt.plot(test, color='red',label='Real')
    plt.plot(predicted, color='blue',label='Predicted')
    plt.title('Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()

def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error is {}.".format(rmse))

In [18]:
y = data['stock_price']
X = data.drop(columns='stock_price')

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [16]:
X_train

,date,symbol,Dividend,volume,fed_funds_rate,gdp,debt_to_equity,eps,return_on_equity,quick_ratio,operating_ratio,inventory_turnover,sa_neu,sa_pos,sa_neg
2423,2019-07-01,WBA,0.00,5.415303e+06,2.40,4796.221000,1.863836,0.878151,0.033433,0.363991,0.683468,1.781430,0.293993,0.499660,0.206347
7090,2021-06-01,CDW,0.00,7.714133e+05,0.08,4907.879667,7.572559,1.981923,0.261346,1.202329,0.934571,4.797569,NaN,NaN,NaN
12674,2023-02-01,XOM,0.91,1.661462e+07,4.57,5002.221667,0.831351,2.926529,0.060109,1.088859,0.723615,2.587264,0.338450,0.000039,0.661511
8483,2021-12-01,CPT,0.83,6.803367e+05,0.08,4940.888333,0.883988,2.060968,0.050720,1.727582,0.337403,14.382010,NaN,NaN,NaN
5909,2020-11-01,NUE,0.00,1.913240e+06,0.09,4792.981333,0.834049,1.088756,0.031089,2.299682,0.909128,1.299067,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,2023-01-01,A,0.00,1.235647e+06,4.33,4950.820000,0.946693,1.189189,0.062756,1.532541,0.785304,0.709271,0.001703,0.993029,0.005268
5191,2020-08-01,JKHY,0.00,5.807800e+05,0.10,4748.625333,0.530804,1.061612,0.052570,1.663751,0.762497,-2.164578,NaN,NaN,NaN
5390,2020-09-01,ATVI,0.00,6.269807e+06,0.09,4796.142667,0.500278,0.782383,0.041997,4.101509,0.628692,14.656250,NaN,NaN,NaN
860,2018-12-01,VLO,0.00,4.786477e+06,2.27,4647.263000,1.314811,2.266667,0.043938,1.039071,1.001914,4.368953,0.693603,0.005750,0.300647
